# core

> Main codebase for fetching and saving RSS feeds

In [ ]:
# | default_exp core

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import json
import logging
from pathlib import Path
from typing import *

import fire
import reader
import rich
from bs4 import BeautifulSoup
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.text import Text

console = Console()

logger = logging.getLogger(__name__)


def strip_html(s: str):
    s = BeautifulSoup(s, "html.parser")
    return s.text

In [ ]:
# | export


class Feed:
    """RSS feed class"""

    def __init__(self, url: str, name: str = None, tags: List[str] = []):
        self.url = url
        self.name = name
        self.tags = tags

    def add_tag(self, tag: str):
        if tag not in self.tags:
            self.tags.append(tag)

    def remove_tag(self, tag: str):
        if tag in self.tags:
            self.tags.remove(tag)

In [ ]:
# | export


class PyNewsReader:
    def __init__(self, feeds=List[Feed]):
        self._dbfolder = Path().home() / ".cache/pynewsreader"
        if not self._dbfolder.exists():
            self._dbfolder.mkdir(parents=True)

        # If custom feed names exist, read them in
        feed_names_path = self._dbfolder / "feed_names.json"
        if feed_names_path.exists():
            with open(self._dbfolder / "feed_names.json", "rt") as myfile:
                self._feed_names = json.load(myfile)
        else:
            self._feed_names = {}

        # If title blacklist exists, read it in
        title_blacklist_path = self._dbfolder / "title_blacklist.json"
        if title_blacklist_path.exists():
            with open(self._dbfolder / "title_blacklist.json", "rt") as myfile:
                self._title_blacklist = json.load(myfile)
        else:
            self._title_blacklist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        # If title whitelist exists, read it in
        title_whitelist_path = self._dbfolder / "title_whitelist.json"
        if title_whitelist_path.exists():
            with open(self._dbfolder / "title_whitelist.json", "rt") as myfile:
                self._title_whitelist = json.load(myfile)
        else:
            self._title_whitelist = []

        self._reader = reader.make_reader(
            self._dbfolder / "db.sqlite",
            plugins=["reader.enclosure_dedupe", "reader.entry_dedupe"],
        )

        self._reader.enable_search()

    def _print_entries(
        self, entries: List[reader.Entry], mark_as_read: bool = True, limit: int = 10
    ):
        """Pretty print entries - supports reader.Reader.get_entries arguments"""
        displayed_links = set()
        for e in entries:
            if e.link in displayed_links:
                # Don't display duplicates
                self._reader.mark_entry_as_read(e)
            else:
                displayed_links.add(e.link)
                if e.published:
                    published_date = "Date: " + e.published.isoformat()[:10]
                else:
                    published_date = "Date: Unknown"
                if mark_as_read:
                    self._reader.mark_entry_as_read(e)

                feed_title = f"[bold]{self._get_feed_title(e.original_feed_url)}[/bold]"

                if e.important:
                    panel_body = ":exclamation_mark:"
                else:
                    panel_body = ""
                panel_body += f"Title: [bold]{e.title}[/bold]" + "\n"
                panel_body += str(published_date) + "\n\n"
                panel_body += strip_html(e.summary).strip() + "\n"

                console.print(
                    Panel(
                        panel_body,
                        title=feed_title,
                        subtitle=f"[link={e.link}]{e.link}[/link]",
                    )
                )
                console.print()
            if len(displayed_links) == limit:
                return

    def _search_to_entry(self, search_result):
        """For a given `reader` search result return the corresponding `reader.Entry`"""
        for i in self._reader.get_entries():
            if i.id == search_result.id and i.feed_url == search_result.feed_url:
                return i

    def _get_feed_title(self, url: str):
        """Get display title for pynewsreader feed"""
        if url in self._feed_names and self._feed_names[url] is not None:
            return self._feed_names[url]
        elif self._reader.get_feed(url).title:
            return self._reader.get_feed(url).title
        else:
            return self._reader.get_feed(url).url

    def _mark_matching_entries_as_read(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as read: {i.title}")
                    self._reader.mark_entry_as_read(i)

    def _mark_matching_entries_as_important(self, match_strings: List):
        for i in self._reader.get_entries(read=False):
            for filter_string in match_strings:
                if filter_string in i.title:
                    print(f"Marking entry as important: {i.title}")
                    self._reader.mark_entry_as_important(i)

    def _get_entries(
        self, important: bool = None, read: Union[None, bool] = None, limit: int = 10
    ):
        """Get entries in reader.Entry format"""
        return self._reader.get_entries(read=read, limit=limit, important=important)

    def _get_tags(self, entry: reader.Entry):
        """Get tags for a given entry"""
        return [i[0] for i in list(self._reader.get_tags(entry))]

    def _add_to_blacklist(self, blacklist_string: str):
        """Add entry to blacklist"""
        if blacklist_string not in self._title_blacklist:
            self._title_blacklist.append(blacklist_string)
            with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
                json.dump(self._title_blacklist, myfile)

    def _remove_from_blacklist(self, blacklist_string: str):
        """Remove entry from blacklist"""
        if blacklist_string in self._title_blacklist:
            self._title_blacklist.remove(blacklist_string)
            with open(self._dbfolder / "title_blacklist.json", "wt") as myfile:
                json.dump(self._title_blacklist, myfile)

    def _add_to_whitelist(self, whitelist_string: str):
        """Add entry to whitelist"""
        if whitelist_string not in self._title_whitelist:
            self._title_whitelist.append(whitelist_string)
            with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
                json.dump(self._title_whitelist, myfile)
            for entry in self._get_entries(limit=None):
                if whitelist_string in entry.title:
                    self._reader.mark_entry_as_important(entry)

    def _remove_from_whitelist(self, whitelist_string: str):
        """Remove whitelist entry"""
        if whitelist_string in self._title_whitelist:
            self._title_whitelist.remove(whitelist_string)
            with open(self._dbfolder / "title_whitelist.json", "wt") as myfile:
                json.dump(self._title_whitelist, myfile)
            for entry in self._get_entries(limit=None):
                if whitelist_string in entry.title:
                    self._reader.mark_entry_as_unimportant(entry)

    def _mark_important(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_important(entry)

    def _mark_unimportant(self, entry: reader.Entry = None):
        """Mark entry as important"""
        if entry is not None:
            reader.Reader.mark_entry_as_unimportant(entry)

    def _add_tag(self, entry: reader.Entry, tag_key: str, tag_value: Dict = None):
        """Add tag to entry"""
        reader.Reader.set_tag(entry, tag_key, tag_value)

    def _remove_tag(self, entry: reader.Entry, tag_key: str):
        """Remove tag from entry"""
        self._reader.delete_tag(entry, tag_key)

    def add_feed(self, feed: Union[Feed, str]):
        """Add feed to pynewsreader

        Args:
            feed (Feed): pynewsreader Feed to add
        """
        if isinstance(feed, Feed):
            self._feed_names[feed.url] = feed.name
            self._reader.add_feed(feed.url, exist_ok=True)
        elif isinstance(feed, str):
            self._reader.add_feed(feed, exist_ok=True)
        else:
            raise Exception("Must be str or Feed type to add")

        # Save names to file
        with open(self._dbfolder / "feed_names.json", "wt") as myfile:
            json.dump(self._feed_names, myfile)

    def remove_feed(self, feed: Feed):
        """Remove feed from pynewsreader instance

        Args:
            feed (Feed): Feed to remove
        """

        self._reader.delete_feed(feed.url)

    def feeds(self):
        """List pynewsreader feeds

        Returns:
            List[str]: List of names of current pynewsreader feeds
        """
        return [self._get_feed_title(i.url) for i in self._reader.get_feeds()]

    def update(self):
        """Update feeds and search"""
        self._reader.update_feeds()
        self._reader.update_search()
        if len(self._title_blacklist) > 0:
            self._mark_matching_entries_as_read(self._title_blacklist)
        if len(self._title_whitelist) > 0:
            self._mark_matching_entries_as_important(self._title_whitelist)

    def show(
        self,
        limit: int = 6,
        read: bool = False,
        important: bool = None,
        mark_as_read: bool = True,
    ):
        """Pretty print entries

        Args:
            limit (int, optional): Number of entries to show. Defaults to 5.
            read (bool, optional): Show read entries (True), unread entries (False), or all entries (None). Defaults to None.
            mark_as_read (bool, optional): Mark displayed entries as read. Defaults to False.
        """
        self._print_entries(
            self._get_entries(read=read, important=important, limit=limit * 2),
            limit=limit,
            mark_as_read=mark_as_read,
        )

    def search(self, query: str, mark_as_read: bool = True, limit: int = 10):
        """Search entries and pretty print results

        Args:
            query (str): Search query
            mark_as_read (bool, optional): Mark results as read? Defaults to True.
        """
        self._print_entries(
            [self._search_to_entry(i) for i in self._reader.search_entries(query)],
            mark_as_read=mark_as_read,
            limit=limit,
        )

In [ ]:
r = PyNewsReader()

## Add Feeds

In [ ]:
# With names
for i in [
    Feed(url="https://ricochet.media/en/feed", name="Richochet Media"),
    Feed(url="https://thetyee.ca/rss2.xml", name="The Tyee"),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
        name="Toronto Star | Top Stories",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
        name="Toronto Star | Investigations",
    ),
    Feed(
        url="https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
        name="Toronto Star | Editorials",
    ),
    Feed(url="https://www.macleans.ca/feed/", name="Macleans"),
]:
    r.add_feed(i)

# Use the default name from the feed
for i in [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://thenarwhal.ca/feed/",
]:
    r.add_feed(Feed(i))

## Update Feeds

In [ ]:
r.update()

## List Feeds

In [ ]:
r.feeds()

['Richochet Media',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans',
 'The Narwhal',
 'The Tyee',
 'Toronto Star | Investigations',
 'Toronto Star | Editorials',
 'Toronto Star | Top Stories']

## Display Entries

`read` = None shows all entries

`read` = False shows only unread entries

`read` = True shows only read entries

In [ ]:
r.show(limit=5, read=None, mark_as_read=False)

╭──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╮
│ Title: Their Pride flag was vandalized 4 times. Now this N.S. business is flooded with support                  │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ The Pride flag of a recently opened business in the Annapolis Valley has been vandalized four times. But the    │
│ businesses owner say after an outpouring support, "the love far outweighs the tiny little bit of hate.”         │
│                                                                                                                 │
╰─ ]8;id=436232;https://www.cbc.ca/news/canada/nova-scotia/their-pride-flag-was-vandalized-4-times-now-this-n-s-business-is-flooded-with-support-1.6699891?cmp=rss\https://www.cbc.ca/news/canada/nova-scotia/their-pride-flag-was-vandalized-4-times-now-this-n-s-business-is-fl]8;;\─╯

╭─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╮
│ Title: Netanyahu sworn in for unprecedented 6th term in Israel after being ousted last year                     │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ Benjamin Netanyahu returned to power Thursday for an unprecedented sixth term as Israel's prime minister,       │
│ taking the helm of the most right-wing and religiously conservative government in the country's 74-year         │
│ history.                                                                                                        │
│                                                                                                                 │
╰──────────────── ]8;id=561372;https://www.cbc.ca/news/world/netanyahu-israel-prime-minister-1.6699880?cmp=rss\https://www.cbc.ca/news/world/netanyahu-israel-prime-minister-1.6699880?cmp=rss]8;;\ ────────────────╯

╭─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╮
│ Title: Vivienne Westwood, fashion designer and punk style legend, dead at 81                                    │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ Vivienne Westwood, an influential fashion maverick who played a key role in the punk movement with her radical  │
│ approach to urban street style, died on Thursday at 81.                                                         │
│                                                                                                                 │
╰──────────────────── ]8;id=484922;https://www.cbc.ca/news/world/vivienne-westwood-obit-1.6699903?cmp=rss\https://www.cbc.ca/news/world/vivienne-westwood-obit-1.6699903?cmp=rss]8;;\ ─────────────────────╯

╭─────────────────────────────────────────────── CBC | Ottawa News ───────────────────────────────────────────────╮
│ Title: Some Hydro One customers could be without power for weeks after snowstorm                                │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ Ontario's largest electricity supplier says it may take weeks to fully restore power in some areas after a      │
│ massive snowstorm hit the province.                                                                             │
│                                                                                                                 │
╰──────────── ]8;id=717187;https://www.cbc.ca/news/canada/toronto/electricity-ontario-snowstorm-1.6699814?cmp=rss\https://www.cbc.ca/news/canada/toronto/electricity-ontario-snowstorm-1.6699814?cmp=rss]8;;\ ─────────────╯

╭─────────────────────────────────────────────── CBC | World News ────────────────────────────────────────────────╮
│ Title: Lack of information on China's COVID-19 surge stirs global concern                                       │
│ Date: 2022-12-29                                                                                                │
│                                                                                                                 │
│ Six countries, including Italy and the U.S., have announced testing requirements for passengers from China, as  │
│ the country continues to be accused of not being forthcoming about the virus.                                   │
│                                                                                                                 │
╰─────── ]8;id=597643;https://www.cbc.ca/news/health/china-covid-global-concern-testing-requirements-1.6699690?cmp=rss\https://www.cbc.ca/news/health/china-covid-global-concern-testing-requirements-1.6699690?cmp=rss]8;;\ ────────╯

## Search Entries

In [ ]:
r.search("rate", mark_as_read=False)

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: High office vacancy rates spell continuing trouble for shops dependent on worker foot traffic            │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ Businesses that depend on the flow of office workers are struggling as vacancy rates in office towers across    │
│ Canada continue to rise. Vancouver's office tower vacancy rate has tripled since 2019, while Montreal's has     │
│ doubled.                                                                                                        │
│                                                                                                                 │
╰────────── ]8;id=930731;https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss\https://www.cbc.ca/news/business/office-vacancy-rates-businesses-struggle-1.6692360?cmp=rss]8;;\ ──────────╯

╭────────────────────────────────────────────── CBC | Business News ──────────────────────────────────────────────╮
│ Title: Official inflation rate cools to 6.8% — but just about everything except gasoline keeps getting more     │
│ expensive                                                                                                       │
│ Date: 2022-12-21                                                                                                │
│                                                                                                                 │
│ Canada's inflation rate cooled to 6.8 per cent last month as prices for gasoline and furniture went down, but   │
│ the cost of food and rent went up.                                                                              │
│                                                                                                                 │
╰────────────────────── ]8;id=296499;https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss\https://www.cbc.ca/news/business/inflation-canada-1.6693441?cmp=rss]8;;\ ──────────────────────╯

╭───────────────────────────────────────── Toronto Star | Investigations ─────────────────────────────────────────╮
│ Title: Toronto parents less likely to see reduced daycare fees than almost anywhere else in Ontario             │
│ Date: 2022-12-22                                                                                                │
│                                                                                                                 │
│ For-profit daycares opted out of $10/day child care program at much higher rate than non-profits, a Star        │
│ analysis found.                                                                                                 │
│                                                                                                                 │
╰─ ]8;id=191363;https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees-than-almost-anywhere-else-in-ontario.html\https://www.thestar.com/news/investigations/2022/12/22/toronto-parents-less-likely-to-see-reduced-daycare-fees]8;;\─╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: Rent or buy? An expert weighs in on the sky-high rental market                                           │
│ Date: 2022-12-05                                                                                                │
│                                                                                                                 │
│ The deputy chief economist with Canada Mortgage and Housing Corporation on Canada’s most expensive places to    │
│ rent and the future of housing                                                                                  │
│ The post Rent or buy? An expert weighs in on the sky-high rental market appeared first on Macleans.ca.          │
│                                                                                                                 │
╰─ ]8;id=60246;https://www.macleans.ca/economy/realestateeconomy/rent-or-buy-an-expert-weighs-in-on-the-sky-high-rental-market/\https://www.macleans.ca/economy/realestateeconomy/rent-or-buy-an-expert-weighs-in-on-the-sky-high-rental-marke]8;;\─╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: My Prediction: Rent-to-own will transform how Canadians buy and sell homes                               │
│ Date: 2022-12-15                                                                                                │
│                                                                                                                 │
│ Archaic rules keep homeownership out of reach for millions. That’s about to change.                             │
│ The post My Prediction: Rent-to-own will transform how Canadians buy and sell homes appeared first on           │
│ Macleans.ca.                                                                                                    │
│                                                                                                                 │
╰─ ]8;id=417803;https://www.macleans.ca/year-ahead/my-prediction-rent-to-own-will-transform-how-canadians-buy-and-sell-homes/\https://www.macleans.ca/year-ahead/my-prediction-rent-to-own-will-transform-how-canadians-buy-and-sell-homes/]8;;\ ─╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: The Year Ahead: Real Estate in 2023                                                                      │
│ Date: 2022-12-06                                                                                                │
│                                                                                                                 │
│ A massive market correction, a NIMBY battle, and rents rise amid questions over supply                          │
│ The post The Year Ahead: Real Estate in 2023 appeared first on Macleans.ca.                                     │
│                                                                                                                 │
╰──────────────────── ]8;id=28206;https://www.macleans.ca/year-ahead/the-year-ahead-real-estate-in-2023/\https://www.macleans.ca/year-ahead/the-year-ahead-real-estate-in-2023/]8;;\ ─────────────────────╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: Top 50 Fastest-growing Charities in Canada                                                               │
│ Date: 2022-12-01                                                                                                │
│                                                                                                                 │
│ Created for                                                                                                     │
│ With its Fastest-growing Charities in Canada annual report, the Veritas Foundation aims to show which of the    │
│ country’s roughly 86,000 charities are attracting the most interest from donors.…                               │
│ The post Top 50 Fastest-growing Charities in Canada appeared first on Macleans.ca.                              │
│                                                                                                                 │
╰───────────────── ]8;id=429465;https://www.macleans.ca/sponsored/top-50-fastest-growing-charities-in-canada/\https://www.macleans.ca/sponsored/top-50-fastest-growing-charities-in-canada/]8;;\ ─────────────────╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: Re-defining innovation in MS—and the power of partnership to get us there                                │
│ Date: 2022-12-06                                                                                                │
│                                                                                                                 │
│ There is a clear need to improve the standard of care for people living with MS. Fostering cross-sector         │
│ collaboration can bring us one step closer.                                                                     │
│ The post Re-defining innovation in MS—and the power of partnership to get us there appeared first on            │
│ Macleans.ca.                                                                                                    │
│                                                                                                                 │
╰───────────────────────── ]8;id=180510;https://www.macleans.ca/sponsored/innovation-in-ms-novartis/\https://www.macleans.ca/sponsored/innovation-in-ms-novartis/]8;;\ ──────────────────────────╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: The Year Ahead: Politics in 2023                                                                         │
│ Date: 2022-12-06                                                                                                │
│                                                                                                                 │
│ Trudeau, Poilievre, and the fight for democracy take centre stage                                               │
│ The post The Year Ahead: Politics in 2023 appeared first on Macleans.ca.                                        │
│                                                                                                                 │
╰────────────────────── ]8;id=257245;https://www.macleans.ca/year-ahead/the-year-ahead-politics-in-2023/\https://www.macleans.ca/year-ahead/the-year-ahead-politics-in-2023/]8;;\ ──────────────────────╯

╭─────────────────────────────────────────────────── Macleans ────────────────────────────────────────────────────╮
│ Title: The Year Ahead: Food in 2023                                                                             │
│ Date: 2022-12-06                                                                                                │
│                                                                                                                 │
│ "It seems patrons do not want a side of exhaust with their brunch"                                              │
│ The post The Year Ahead: Food in 2023 appeared first on Macleans.ca.                                            │
│                                                                                                                 │
╰──────────────────── ]8;id=562693;https://www.macleans.ca/year-ahead/year-ahead-food-restaurants-dinner/\https://www.macleans.ca/year-ahead/year-ahead-food-restaurants-dinner/]8;;\ ─────────────────────╯

## Tag Entry

In [ ]:
test = list(r._reader.get_entries())

In [ ]:
r._reader.set_tag(test[0], "foobar")

In [ ]:
list(r._reader.get_tags(test[0]))

[('foobar', None)]

In [ ]:
# We added a method to just return the tag key:
r._get_tags(test[0])

['foobar']

## Remove tag from entry

In [ ]:
r._reader.delete_tag(test[0], "foobar")

In [ ]:
list(r._get_tags(test[0]))

[]

## Mark as Important/Unimportant

In [ ]:
r._reader.mark_entry_as_important(test[0])
r._reader.mark_entry_as_unimportant(test[0])

## Mark as Read / Unread

In [ ]:
r._reader.mark_entry_as_read(test[0])
r._reader.mark_entry_as_unread(test[0])

## Automatically mark entries as read

When updating feeds, entries with titles matching these strings will automatically be marked as read

In [ ]:
for i in ["Musk", "Apple", "Bezos", "Google", "Samsung", "iPhone", "iPad"]:
    r._add_to_blacklist(i)

In [ ]:
r._title_blacklist

['Musk', 'Apple', 'Bezos', 'Google', 'Samsung', 'iPhone', 'iPad', 'Canada']

In [ ]:
r.update()

## Automatically mark entries as important

In [ ]:
for i in ["interest rate", "Bank of Canada", "housing market"]:
    r._add_to_whitelist(i)

In [ ]:
r.update()

## CLI

In [ ]:
# | export


def main():
    fire.Fire(PyNewsReader)

In [ ]:
# | hide

# To Do:
# * User interface

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()